In [1]:
# 亲和性分析
# 用Apriori 算法挖掘关联特征， 做电影推荐

# data from http://grouplens.org/datasets/movielens/

In [2]:
import os
data_folder = os.path.join(os.path.expanduser("~"), "workspace\Learning-Data-Mining-with-Python\Chapter 4", "Data")
ratings_filename = os.path.join(data_folder, "ratings.csv")

In [3]:
import pandas as pd

In [14]:
# if no header, can add header as below
#all_ratings = pd.read_csv(ratings_filename, delimiter="\t", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])
all_ratings = pd.read_csv(ratings_filename)
all_ratings["timestamp"] = pd.to_datetime(all_ratings['timestamp'],unit='s')
all_ratings[:5]

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [18]:
# As you can see, there are no review for most movies, such as #213
all_ratings[all_ratings["userId"] == 675].sort_values("movieId")

,userId,movieId,rating,timestamp
96960,675,3,3.0,1997-06-13 14:57:14
96961,675,6,4.0,1997-06-13 14:57:14
96962,675,7,1.0,1997-06-13 14:58:44
96963,675,16,3.0,1997-06-17 17:28:59
96964,675,17,3.0,1997-06-13 14:55:36
96965,675,22,3.0,1997-06-17 17:35:03
96966,675,24,4.0,1997-06-17 17:35:03
96967,675,25,4.0,1997-06-13 14:55:37
96968,675,32,4.0,1997-06-13 14:55:33
96969,675,34,3.0,1997-06-17 17:36:12


In [20]:
# Not all reviews are favourable! Our goal is "other recommended books", so we only want favourable reviews
all_ratings["favorable"] = all_ratings["rating"] > 3
all_ratings[10:15]

,userId,movieId,rating,timestamp,favorable
10,1,293,4.0,2005-04-02 23:31:43,True
11,1,296,4.0,2005-04-02 23:32:47,True
12,1,318,4.0,2005-04-02 23:33:18,True
13,1,337,3.5,2004-09-10 03:08:29,True
14,1,367,3.5,2005-04-02 23:53:00,True


In [22]:
all_ratings[all_ratings["userId"] == 1][:5]

,userId,movieId,rating,timestamp,favorable
0,1,2,3.5,2005-04-02 23:53:47,True
1,1,29,3.5,2005-04-02 23:31:16,True
2,1,32,3.5,2005-04-02 23:33:39,True
3,1,47,3.5,2005-04-02 23:32:07,True
4,1,50,3.5,2005-04-02 23:29:40,True


In [23]:
# Sample the dataset. You can try increasing the size of the sample, but the run time will be considerably longer
ratings = all_ratings[all_ratings['userId'].isin(range(200))]  # & ratings["UserID"].isin(range(100))]

In [24]:
ratings

,userId,movieId,rating,timestamp,favorable
0,1,2,3.5,2005-04-02 23:53:47,True
1,1,29,3.5,2005-04-02 23:31:16,True
2,1,32,3.5,2005-04-02 23:33:39,True
3,1,47,3.5,2005-04-02 23:32:07,True
4,1,50,3.5,2005-04-02 23:29:40,True
5,1,112,3.5,2004-09-10 03:09:00,True
6,1,151,4.0,2004-09-10 03:08:54,True
7,1,223,4.0,2005-04-02 23:46:13,True
8,1,253,4.0,2005-04-02 23:35:40,True
9,1,260,4.0,2005-04-02 23:33:46,True


In [26]:
# We start by creating a dataset of each user's favourable reviews
favorable_ratings = ratings[ratings["favorable"]]
favorable_ratings[:5]

,userId,movieId,rating,timestamp,favorable
0,1,2,3.5,2005-04-02 23:53:47,True
1,1,29,3.5,2005-04-02 23:31:16,True
2,1,32,3.5,2005-04-02 23:33:39,True
3,1,47,3.5,2005-04-02 23:32:07,True
4,1,50,3.5,2005-04-02 23:29:40,True


In [34]:
# We are only interested in the reviewers who have more than one review
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("userId")["movieId"])
len(favorable_reviews_by_users)
#favorable_reviews_by_users[2]

199

In [36]:
# Find out how many movies have favourable ratings
num_favorable_by_movie = ratings[["movieId", "favorable"]].groupby("movieId").sum()
num_favorable_by_movie.sort_values("favorable", ascending=False)[:5]

,favorable
movieId,
296,80.0
356,78.0
318,76.0
593,63.0
480,58.0


In [38]:
from collections import defaultdict
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [39]:
import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 50

# k=1 candidates are the isbns with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["favorable"])
                                for movie_id, row in num_favorable_by_movie.iterrows()
                                if row["favorable"] > min_support)

print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

There are 11 movies with more than 50 favorable reviews
I found 34 frequent itemsets of length 2
I found 49 frequent itemsets of length 3
I found 36 frequent itemsets of length 4
I found 12 frequent itemsets of length 5
I found 1 frequent itemsets of length 6
Did not find any frequent itemsets of length 7


In [40]:
print("Found a total of {0} frequent itemsets".format(sum(len(itemsets) for itemsets in frequent_itemsets.values())))

Found a total of 132 frequent itemsets


In [41]:
# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

There are 425 candidate rules


In [42]:
print(candidate_rules[:5])

[(frozenset({47}), 50), (frozenset({50}), 47), (frozenset({47}), 296), (frozenset({296}), 47), (frozenset({47}), 318)]


In [43]:
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

In [44]:
# Choose only rules above a minimum confidence level
min_confidence = 0.9

In [45]:
# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

36


In [49]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)
sorted_confidence

[((frozenset({47, 356, 593}), 296), 1.0),
 ((frozenset({50, 318, 2858}), 296), 1.0),
 ((frozenset({50, 593, 2858}), 296), 1.0),
 ((frozenset({50, 356, 593}), 296), 1.0),
 ((frozenset({50, 318, 593, 2858}), 296), 1.0),
 ((frozenset({47, 50, 356, 593}), 296), 1.0),
 ((frozenset({47, 318, 356, 593}), 296), 1.0),
 ((frozenset({50, 318, 356, 593}), 296), 1.0),
 ((frozenset({47, 50, 318, 356, 593}), 296), 1.0),
 ((frozenset({47, 593}), 296), 0.9705882352941176),
 ((frozenset({47, 318, 593}), 296), 0.9642857142857143),
 ((frozenset({50, 2858}), 296), 0.9615384615384616),
 ((frozenset({47, 50, 593}), 296), 0.96),
 ((frozenset({318, 593, 2858}), 296), 0.96),
 ((frozenset({47, 50, 318, 593}), 296), 0.9565217391304348),
 ((frozenset({356, 593, 2858}), 296), 0.95),
 ((frozenset({356, 593, 2858}), 318), 0.95),
 ((frozenset({318, 356, 593, 2858}), 296), 0.9473684210526315),
 ((frozenset({296, 356, 593, 2858}), 318), 0.9473684210526315),
 ((frozenset({50, 527, 593}), 296), 0.9444444444444444),
 ((fro

In [50]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise, conclusion))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends frozenset({593, 356, 47}) they will also recommend 296
 - Confidence: 1.000

Rule #2
Rule: If a person recommends frozenset({50, 2858, 318}) they will also recommend 296
 - Confidence: 1.000

Rule #3
Rule: If a person recommends frozenset({593, 50, 2858}) they will also recommend 296
 - Confidence: 1.000

Rule #4
Rule: If a person recommends frozenset({593, 50, 356}) they will also recommend 296
 - Confidence: 1.000

Rule #5
Rule: If a person recommends frozenset({593, 2858, 50, 318}) they will also recommend 296
 - Confidence: 1.000



In [51]:
# Even better, we can get the movie titles themselves from the dataset
movie_name_filename = os.path.join(data_folder, "movies.csv")
#movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")
#movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure",
#                           "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir",
#                           "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
movie_name_data = pd.read_csv(movie_name_filename,encoding = "mac-roman")

In [52]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["movieId"] == movie_id]["title"]
    title = title_object.values[0]
    return title

In [53]:
get_movie_name(4)

'Waiting to Exhale (1995)'

In [54]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Forrest Gump (1994), Seven (a.k.a. Se7en) (1995) they will also recommend Pulp Fiction (1994)
 - Confidence: 1.000

Rule #2
Rule: If a person recommends Usual Suspects, The (1995), American Beauty (1999), Shawshank Redemption, The (1994) they will also recommend Pulp Fiction (1994)
 - Confidence: 1.000

Rule #3
Rule: If a person recommends Silence of the Lambs, The (1991), Usual Suspects, The (1995), American Beauty (1999) they will also recommend Pulp Fiction (1994)
 - Confidence: 1.000

Rule #4
Rule: If a person recommends Silence of the Lambs, The (1991), Usual Suspects, The (1995), Forrest Gump (1994) they will also recommend Pulp Fiction (1994)
 - Confidence: 1.000

Rule #5
Rule: If a person recommends Silence of the Lambs, The (1991), American Beauty (1999), Usual Suspects, The (1995), Shawshank Redemption, The (1994) they will also recommend Pulp Fiction (1994)
 - Confidence: 1.000



In [56]:
# Evaluation using test data
test_dataset = all_ratings[~all_ratings['userId'].isin(range(200))]
test_favorable = test_dataset[test_dataset["favorable"]]
#test_not_favourable = test_dataset[~test_dataset["Favourable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("userId")["movieId"])
#test_not_favourable_by_users = dict((k, frozenset(v.values)) for k, v in test_not_favourable.groupby("UserID")["MovieID"])
#test_users = test_dataset["UserID"].unique()

In [57]:
test_dataset[:5]

,userId,movieId,rating,timestamp,favorable
25048,200,6,5.0,1996-08-11 12:59:30,True
25049,200,10,3.0,1996-08-11 12:53:11,False
25050,200,17,4.0,1996-08-11 12:57:25,True
25051,200,19,2.0,1996-08-11 12:54:08,False
25052,200,20,4.0,1996-08-11 13:05:27,True


In [58]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [59]:
test_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

36


In [60]:
sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)
print(sorted_test_confidence[:5])

[((frozenset({356, 47, 593, 50, 318}), 296), 0.8967280163599182), ((frozenset({593, 50, 356, 47}), 296), 0.8868425712767841), ((frozenset({593, 50, 318, 47}), 296), 0.8849057986951103), ((frozenset({593, 2858, 50, 318}), 296), 0.8784590798216839), ((frozenset({356, 296, 47, 593, 50}), 318), 0.8755990415335463)]


In [61]:
for index in range(10):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

Rule #1
Rule: If a person recommends Silence of the Lambs, The (1991), Forrest Gump (1994), Seven (a.k.a. Se7en) (1995) they will also recommend Pulp Fiction (1994)
 - Train Confidence: 1.000
 - Test Confidence: 0.827

Rule #2
Rule: If a person recommends Usual Suspects, The (1995), American Beauty (1999), Shawshank Redemption, The (1994) they will also recommend Pulp Fiction (1994)
 - Train Confidence: 1.000
 - Test Confidence: 0.845

Rule #3
Rule: If a person recommends Silence of the Lambs, The (1991), Usual Suspects, The (1995), American Beauty (1999) they will also recommend Pulp Fiction (1994)
 - Train Confidence: 1.000
 - Test Confidence: 0.857

Rule #4
Rule: If a person recommends Silence of the Lambs, The (1991), Usual Suspects, The (1995), Forrest Gump (1994) they will also recommend Pulp Fiction (1994)
 - Train Confidence: 1.000
 - Test Confidence: 0.841

Rule #5
Rule: If a person recommends Silence of the Lambs, The (1991), American Beauty (1999), Usual Suspects, The (1995)